# Brazilian cities to travel

<b> Goal: </b> This portfolio project showcases our expertise in data analysis through an in-depth examination of Brazilian cities identified by the Brazilian Ministry of Tourism as key destinations in their public policy plan (Mapa do Turismo/Tourism Map)

In [ ]:
## adicionar índice [[indice]]

### 1. Descriptive Analysis

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

## 1.1 Categories

In [3]:
df = pd.read_excel(r"data\relatorio_categorizacao_2019-portal.xls")

df.head()

,MACRO,UF,REGIAO_TURISTICA,MUNICIPIO,COD_IBGE,QUANTIDADE_EMPREGOS,QUANTIDADE_ ESTABELECIMENTOS,QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL,QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL,ARRECADACAO,CLUSTER
0,Centro-Oeste,MT,Região Turística Portal do Agronegócio,Nova Mutum,5106224,89,14,0,15309,708887,C
1,Nordeste,MA,Polo Lagos e Campos Floridos,Cajapió,2102408,0,0,0,1376,0,D
2,Sudeste,MG,Caminhos da Mantiqueira,Brazópolis,3108909,6,3,0,25587,0,D
3,Sudeste,MG,Ouro,Caeté,3110004,310,11,0,23757,4944942,B
4,Sudeste,MG,Pedras Preciosas,Catuji,3115458,0,0,149,0,0,D


In [5]:
#showcasing data types and non-null cells in each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2694 entries, 0 to 2693
Data columns (total 11 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   MACRO                                        2694 non-null   object
 1   UF                                           2694 non-null   object
 2   REGIAO_TURISTICA                             2694 non-null   object
 3   MUNICIPIO                                    2694 non-null   object
 4   COD_IBGE                                     2694 non-null   int64 
 5   QUANTIDADE_EMPREGOS                          2694 non-null   int64 
 6   QUANTIDADE_ ESTABELECIMENTOS                 2694 non-null   int64 
 7   QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL  2694 non-null   int64 
 8   QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL       2694 non-null   int64 
 9   ARRECADACAO                                  2694 non-null   int64 
 10  CLUSTER     

In [ ]:
## pie chart: distribuição por estado
## stacked bar chart: distribuição de cluster por estado

In [6]:
#seeing most numbers
df.describe()

,COD_IBGE,QUANTIDADE_EMPREGOS,QUANTIDADE_ ESTABELECIMENTOS,QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL,QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL,ARRECADACAO
count,2.694000e+03,2694.000000,2694.000000,2.694000e+03,2.694000e+03,2.694000e+03
mean,3.405169e+06,93.380475,7.516333,3.850519e+03,6.383554e+04,1.238121e+06
std,9.310852e+05,555.239839,26.203060,4.667117e+04,3.930170e+05,1.310179e+07
min,1.100015e+06,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.905852e+06,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.302650e+06,6.000000,2.000000,3.800000e+01,5.376500e+03,0.000000e+00
75%,4.205982e+06,35.000000,6.000000,3.730000e+02,2.789175e+04,0.000000e+00
max,5.300108e+06,17946.000000,785.000000,1.651696e+06,1.347863e+07,4.733092e+08


[[ANÁLISE SOBRE:
- 25%-50% de demanda internacional e nacional é zero ((fazer boxplots))
- a distância de mediana e mean em todas as métricas
- entender o perfil de cada categoria/classe
- entender a distribuição de 0 -- provavelmente todos os null foram convertidos em 0]]

In [ ]:
##boxplot de todas as métricas
## count 0 (possi)

## 1.2 Histórico

avaliar quais métricas conseguem ser cruzadas com o histórico 2016 e 2017.